In [1]:
import os
import re
from PIL import Image
import numpy as np
import keras
import pandas as pd
import keras.optimizers as op
initializations = keras.initializers

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#数据采集处理
data_x = []
data_y = []
for root, dirs, files in os.walk('input_data', topdown=False):
    for name in files:
        filepath = os.path.join(root, name)
        image = Image.open(filepath)
        image = image.convert('L')
        image_arr = np.array(image)
        label = re.findall(r'/(.+?)/',filepath)[0]
        data_x.append(image_arr)
        data_y.append(label)
np.savez('data.npz',data_x,data_y)

In [3]:
#获取数据把数据分为训练集，测试集
data = np.load('data.npz')
X,y = data['arr_0'],data['arr_1']
y = pd.get_dummies(y)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X/255,y,test_size=0.2,random_state=1)

In [4]:
from keras.layers import Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(** kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        # W.shape = (time_steps, time_steps)
        self.W = self.add_weight(name='att_weight',
                                 shape=(input_shape[1], input_shape[1]),
                                 initializer='uniform',
                                 trainable=True)
        self.b = self.add_weight(name='att_bias', 
                                 shape=(input_shape[1],),
                                 initializer='uniform',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        # inputs.shape = (batch_size, time_steps, seq_len)
        x = K.permute_dimensions(inputs, (0, 2, 1))
        # x.shape = (batch_size, seq_len, time_steps)
        a = K.softmax(K.tanh(K.dot(x, self.W) + self.b))
        outputs = K.permute_dimensions(a * x, (0, 2, 1))
        outputs = K.sum(outputs, axis=1)
        return outputs

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

In [39]:
from keras.layers import *
from keras.models import *
MAX_SENT_LENGTH = 64*64
review_input = Input(shape=(64,64), dtype='float32')
x = Conv1D(160,2,activation='relu')(review_input)
x = Dropout(0.5)(x)
x = MaxPool1D(2)(x)
x = Conv1D(64,2,activation='relu',padding='same')(x)
x = Dropout(0.5)(x)
x = MaxPool1D(2)(x)
conv = x.get_shape
x = Reshape((15,-1))(x)
# review_encoder = TimeDistributed(sentEncoder)(review_input)  # 对文档中每个句子
l_lstm_sent = Bidirectional(GRU(1000, return_sequences=True),merge_mode='sum')(x)
l_dense_sent = TimeDistributed(Dense(200))(l_lstm_sent)
l_att_sent = AttentionLayer()(l_lstm_sent)
l_att_sent = Dropout(0.6)(l_att_sent)
preds = Dense(4, activation='softmax')(l_att_sent)
model = Model(review_input, preds)
model.compile(loss='categorical_crossentropy',optimizer=op.Adam(lr=1e-4),
              metrics=['accuracy'])

In [44]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
2429/2429 [==============================] - 24s 10ms/step - loss: 1.2371 - acc: 0.4302
Epoch 2/5
2429/2429 [==============================] - 24s 10ms/step - loss: 1.2289 - acc: 0.4327
Epoch 3/5
2429/2429 [==============================] - 24s 10ms/step - loss: 1.2275 - acc: 0.4413
Epoch 4/5
2429/2429 [==============================] - 24s 10ms/step - loss: 1.2149 - acc: 0.4520
Epoch 5/5
2429/2429 [==============================] - 24s 10ms/step - loss: 1.2127 - acc: 0.4389


In [45]:
model.evaluate(x_test,y_test)

608/608 [==============================] - 1s 2ms/step


[1.260695400990938, 0.4095394736842105]

In [71]:
l_att_sent.get_shape()

TensorShape([Dimension(None), Dimension(200)])

In [97]:
#
def attention_3d_block(inputs, single_attention_vector=False):
    # 如果上一层是LSTM，需要return_sequences=True
    # inputs.shape = (batch_size, time_steps, input_dim)
    time_steps = K.int_shape(inputs)[1]
    input_dim = K.int_shape(inputs)[2]
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    if single_attention_vector:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)

    a_probs = Permute((2, 1))(a)
    # 乘上了attention权重，但是并没有求和，好像影响不大
    # 如果分类任务，进行Flatten展开就可以了
    # element-wise
    output_attention_mul = Multiply()([inputs, a_probs]) 
    return output_attention_mul


In [143]:
sentence_input = Input(shape=(64,64), dtype='float32')
l_lstm = Bidirectional(GRU(100, return_sequences=True),merge_mode='sum')(sentence_input)
l_lstm = attention_3d_block(l_lstm)
l_lstm = Flatten()(l_lstm)
l_att_sent = Dropout(0.4)(l_lstm)
preds = Dense(4, activation='softmax')(l_att_sent)
model = Model(sentence_input, preds)

model.compile(loss='categorical_crossentropy',optimizer=op.Adam(lr=1e-4),
              metrics=['accuracy'])

In [202]:
model.fit(x_train,y_train,epochs=500)

Epoch 1/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6712 - acc: 0.7538
Epoch 2/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6856 - acc: 0.7427
Epoch 3/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6522 - acc: 0.7608
Epoch 4/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6259 - acc: 0.7859
Epoch 5/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6407 - acc: 0.7723
Epoch 6/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6493 - acc: 0.7592
Epoch 7/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6365 - acc: 0.7711
Epoch 8/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6207 - acc: 0.7937
Epoch 9/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6515 - acc: 0.7522
Epoch 10/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.6166 - acc: 0.7822
Epoch 11/500
2429/2

2429/2429 [==============================] - 4s 2ms/step - loss: 0.3710 - acc: 0.8810
Epoch 166/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.3435 - acc: 0.8995
Epoch 167/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.3413 - acc: 0.8967
Epoch 168/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.3615 - acc: 0.8876
Epoch 169/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.3437 - acc: 0.8971
Epoch 170/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.3307 - acc: 0.9090
Epoch 171/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.3671 - acc: 0.8860
Epoch 172/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.4905 - acc: 0.8300
Epoch 173/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.7358 - acc: 0.7340
Epoch 174/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.7530 - acc: 0.7225
Epoch 175/500


2429/2429 [==============================] - 4s 2ms/step - loss: 0.2534 - acc: 0.9251
Epoch 248/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.2743 - acc: 0.9234
Epoch 249/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.2760 - acc: 0.9251
Epoch 250/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.4706 - acc: 0.8250
Epoch 251/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.3022 - acc: 0.9103
Epoch 252/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.2438 - acc: 0.9329
Epoch 253/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.2287 - acc: 0.9440
Epoch 254/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.2182 - acc: 0.9452
Epoch 255/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.2188 - acc: 0.9485
Epoch 256/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.2261 - acc: 0.9444
Epoch 257/500


2429/2429 [==============================] - 4s 2ms/step - loss: 0.3226 - acc: 0.8950
Epoch 330/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.3109 - acc: 0.8958
Epoch 331/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1918 - acc: 0.9551
Epoch 332/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1969 - acc: 0.9457
Epoch 333/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1708 - acc: 0.9642
Epoch 334/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1650 - acc: 0.9642
Epoch 335/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1763 - acc: 0.9539
Epoch 336/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1649 - acc: 0.9658
Epoch 337/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1665 - acc: 0.9638
Epoch 338/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1664 - acc: 0.9629
Epoch 339/500


2429/2429 [==============================] - 4s 2ms/step - loss: 0.2458 - acc: 0.9238
Epoch 412/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1576 - acc: 0.9617
Epoch 413/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1274 - acc: 0.9749
Epoch 414/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1183 - acc: 0.9782
Epoch 415/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1149 - acc: 0.9811
Epoch 416/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1118 - acc: 0.9798
Epoch 417/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1140 - acc: 0.9778
Epoch 418/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1156 - acc: 0.9765
Epoch 419/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1770 - acc: 0.9514
Epoch 420/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.2662 - acc: 0.9045
Epoch 421/500


2429/2429 [==============================] - 4s 2ms/step - loss: 0.0909 - acc: 0.9819
Epoch 494/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.0904 - acc: 0.9819
Epoch 495/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1145 - acc: 0.9667
Epoch 496/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1256 - acc: 0.9671
Epoch 497/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1015 - acc: 0.9782
Epoch 498/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1035 - acc: 0.9753
Epoch 499/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1101 - acc: 0.9749
Epoch 500/500
2429/2429 [==============================] - 4s 2ms/step - loss: 0.1046 - acc: 0.9782


In [203]:
model.evaluate(x_test,y_test)

608/608 [==============================] - 0s 474us/step


[2.3542112049303556, 0.39473684210526316]